# 0.0 Imports

In [1]:
import pandas             as pd
import numpy              as np
import pandas.io.sql      as sqlio

import matplotlib.pyplot  as plt
import seaborn            as sns

%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import psycopg2

## 0.1 Helper Functions

In [2]:
def create_connection_database(host_name, port_num, database_name, user_name, password_name):
    connection = None
    connection = psycopg2.connect(
                        host = host_name,
                        port = port_num,
                        database = database_name,
                        user = user_name,
                        password = password_name)
    return connection


## 0.2 Loading Data

In [ ]:
#connect to the DB
host_name     = "comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com"
port_num      = 5432 
database_name = "comunidadedsdb"
user_name     = "member"
password_name = "cdspa"

#make connection with DB
conn = create_connection_database (host_name, port_num, database_name, user_name, password_name)

#checking tables in schema
cur = conn.cursor()

cur.execute("""SELECT table_schema, table_name
                      FROM information_schema.tables
                      WHERE table_schema != 'pg_catalog'
                      AND table_schema != 'information_schema'
                      AND table_type='BASE TABLE'
                      ORDER BY table_schema, table_name""")

schemas_db = cur.fetchall()

# cursor close
cur.close()

df_insurance = sqlio.read_sql_query("Select * from pa004.insurance", conn)
df_vehicle = sqlio.read_sql_query("Select * from pa004.vehicle", conn)
df_users = sqlio.read_sql_query("Select * from pa004.users", conn)

#close the connection
conn.close()

#checking each shape of dataframes
print('Tables shape:\n Insurance: {}\n Vehicle: {}\n Users: {}'.format(df_insurance.shape,df_vehicle.shape, df_users.shape ))

In [ ]:
df0 = df_users.merge(df_vehicle, how = 'outer', on='id')
df0 = df0.merge(df_insurance, how = 'outer', on='id')

#cleaning memory of dataframes used to collect data from DB
df_vehicle = None
df_users = None
df_insurance = None

df0.sample(5)

# 1.0 Data Description

In [ ]:
df1 = df0.copy()

## 1.1 Rename Columns

In [ ]:
cols = df1.columns
cols

- Columns names already in snake case

## 1.2 Data Dimension

In [ ]:
print('Number of Rows: {}'.format(df1.shape[0]))
print('Number of Columns: {}'.format(df1.shape[1]))

## 1.3 Data Types

In [ ]:
df1['region_code'] = df1['region_code'].astype(np.int64)
df1['policy_sales_channel'] = df1['policy_sales_channel'].astype(np.int64)
df1.dtypes

- Types are OK:
    - Only numbers (int and float) and categories (object). There is no dates to convert.

## 1.4 Check NAN

In [ ]:
df1.isna().sum()

- There is no NAN in original datas

## 1.5 Fillout NA

Not necessary, as seen in 1.4 section

## 1.6 Columns Description

In [ ]:
cols = df1.columns
cols


- **Id**: Customer identifier 
- **Gender**: Gender of customer
- **Age**: Age of customer
- **Driving License**: 0 if customer does not have the permission for drive; 1 if customer has the permission for drive
- **Region Code**: Region code of customer
- **Previously Insured**: 0 if customer does not have a vehicle insurance; 1 if customer already has a vehicle insurance.
- **Vehicle Age**: Age of vehicle
- **Vehicle Damage**: 0 if customer never got involved in a previous vehicle damage; 1 if customer already had a vehicle damage
- **Annual Premium**: amount paid for health insurance to the company anualy.
- **Policy sales channel**: anonymous code for contact with customer.
- **Vintage**: time (in days) that customer bought his health assurance.
- **Response**: 0 if customer has no interest in product; 1 if customer has interest in product

## 1.7 Descriptive Statistical

In [ ]:
num_attributes = df1[['age', 'annual_premium', 'vintage', 'region_code', 'policy_sales_channel']]

cat_attributes = df1[['driving_license', 'previously_insured', 'response', 'vehicle_damage', 'vehicle_age', 'gender' ]]

### 1.7.1 Numerical Attributes

In [ ]:
# Central Tendency - Mean, median

ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

# Dispersion - std, min, max, range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(min)).T
d3 = pd.DataFrame(num_attributes.apply(max)).T
d4 = pd.DataFrame(num_attributes.apply( lambda x: x.max()- x.min() ) ).T
d5 = pd.DataFrame(num_attributes.apply( lambda x: x.skew() ) ).T
d6 = pd.DataFrame(num_attributes.apply( lambda x: x.kurtosis() ) ).T

#concatenate

m = pd.concat ([d2, d3, d4, ct1, ct2, d1, d5, d6]).T.reset_index()
m.columns = (['attributes', 'min', 'max', 'range', 'mean', 'median', 'std', 'skew', 'kurtosis'])
m

In [ ]:
fig, axs = plt.subplots(ncols = 3, nrows = 2,  figsize = (16,8))
sns.distplot(df1['age'], ax=axs[0][0], bins = 6)
sns.distplot(df1[df1['annual_premium']<100000]['annual_premium'], ax=axs[0][1])
sns.distplot(df1['vintage'], ax=axs[0][2])

sns.distplot(df1['region_code'], ax=axs[1][0])
sns.distplot(df1['policy_sales_channel'], ax=axs[1][1])

In [ ]:
bins = pd.cut(df1['age'], [19, 30, 40, 50, 60, 70, 80, 90])

df1.groupby(bins)['age'].size().sum()
print('Age analysis:')
print('Percent of 20-29: {}'.format(round((df1.groupby(bins)['age'].size()[0]/df1.groupby(bins)['age'].size().sum()),3)))
print('Percent of 30-39: {}'.format(round((df1.groupby(bins)['age'].size()[1]/df1.groupby(bins)['age'].size().sum()),3)))
print('Percent of 40-49: {}'.format(round((df1.groupby(bins)['age'].size()[2]/df1.groupby(bins)['age'].size().sum()),3)))
print('Percent of 50-59: {}'.format(round((df1.groupby(bins)['age'].size()[3]/df1.groupby(bins)['age'].size().sum()),3)))
print('Percent of 60-69: {}'.format(round((df1.groupby(bins)['age'].size()[4]/df1.groupby(bins)['age'].size().sum()),3)))
print('Percent of 70-79: {}'.format(round((df1.groupby(bins)['age'].size()[5]/df1.groupby(bins)['age'].size().sum()),3)))
print('Percent of 80-85: {}'.format(round((df1.groupby(bins)['age'].size()[6]/df1.groupby(bins)['age'].size().sum()),4)))

**Data Description Summary:**

1. Age 
    - Range is from 20 to 85. 
    - Mean is 38.82 and median is 36. 
    - Higher density is from 20 to 29 with 42%, followed by 40 to 49 with 19,9%.
2. Annual_premium
    - Range is from 2630 to 540165.
    - There is a peak in 2630, and a high density around 20000 to 60000.
3. Vintage
    - Range is from 10 to 299.
    - Mean and median are 154. 
    - 

### 1.7.2 Categorical Attributes

In [ ]:
cat_attributes.apply(lambda x: x.unique().shape[0])

In [ ]:
fig, axs = plt.subplots(ncols = 3, nrows = 2,  figsize = (16,8))

sns.countplot(x ='driving_license', data= df1, ax=axs[0][0])
sns.countplot(x = 'previously_insured', data=df1, ax=axs[0][1])
sns.countplot(x = 'vehicle_damage', data=df1, ax=axs[0][2])

sns.countplot(x = 'vehicle_age', data=df1,ax=axs[1][0])
sns.countplot(x = 'gender', data=df1,ax=axs[1][1])
sns.countplot(x = 'response', data=df1, ax=axs[1][2])

- Based on response descriptive statistical analysis, this variable (our target) is unbalanced. This problem is not about an unbalanced problem as default. A balance techinic will be applied to solve this problem.

# 2.0 Feature Engineering And Variables Filtering

In [ ]:
df2 = df1.copy()

vehicle_damage_mapping = {'No':0,
                          'Yes':1}
df2['vehicle_damage'] = df2['vehicle_damage'].map(vehicle_damage_mapping)

In [ ]:
#Creating new features

#annual_premium paid per day
df2['annual_premium_per_day'] = df2['annual_premium']/df2['vintage']

#annual_premium divided per age
df2['annual_premium_per_age'] = df2['annual_premium']/df2['age']

#vintage_per_age
df2['vintage_per_age'] = df2['vintage']/df2['age']

#logic between previously_insured  and vehicle_damage
df2['previously_insured_vehicle_damage'] =  df2.apply(lambda row: -(row['vehicle_damage'] + row['previously_insured'])**2 if row['previously_insured'] == 0 else (row['vehicle_damage'] + row['previously_insured'])**2, axis=1)

#logic between vehicle_age (consider age > 1 as 1 and age < 1 as 0) and vehicle_damage
df2['vehicle_age_vehicle_damage'] = df2.apply(lambda row: -(row['vehicle_damage'] + 0)**2 if row['vehicle_age'] == '<1 Year' else (row['vehicle_damage'] + 1)**2, axis=1)

#logic between driving_license and vehicle_damage
df2['vehicle_damage_license'] = df2.apply(lambda row: -(row['vehicle_damage'] + row['driving_license'])**2 if row['driving_license'] == 0 else (row['vehicle_damage'] + row['driving_license'])**2, axis=1)

#logic between annual_premium divided per mean of annual_premium per region_code
df2_annual_premium_mean_region_code = df2.rename(columns={'annual_premium': 'annual_premium_mean'}).groupby('region_code').mean()
df2_annual_premium_mean_region_code.reset_index(inplace=True)
df2 = df2.merge(df2_annual_premium_mean_region_code[['region_code', 'annual_premium_mean']], on='region_code', how='left')
df2['annual_premium_per_ap_mean_rc'] = df2.apply(lambda row: (row['annual_premium']/row['annual_premium_mean']) ,axis=1)
df2.drop(columns='annual_premium_mean', inplace = True)
df2_annual_premium_mean_region_code = None

#logic between annual_premium divided per mean of annual_premium per policy_sales_channel
df2_annual_premium_mean_policy_sales_channel = df2.rename(columns={'annual_premium': 'annual_premium_mean'}).groupby('policy_sales_channel').mean()
df2_annual_premium_mean_policy_sales_channel.reset_index(inplace=True)
df2 = df2.merge(df2_annual_premium_mean_policy_sales_channel[['policy_sales_channel', 'annual_premium_mean']], on='policy_sales_channel', how='left')
df2['annual_premium_per_ap_mean_psc'] = df2.apply(lambda row: (row['annual_premium']/row['annual_premium_mean']) ,axis=1)
df2.drop(columns='annual_premium_mean', inplace = True)
df2_annual_premium_mean_policy_sales_channel = None

#logic of vehicle_damage_mean per region_code
df2_vehicle_damage_mean_per_region_code = df2.rename(columns={'vehicle_damage': 'vehicle_damage_mean_region_code'}).groupby('region_code').mean()
df2_vehicle_damage_mean_per_region_code.reset_index(inplace=True)
df2 = df2.merge(df2_vehicle_damage_mean_per_region_code[['region_code', 'vehicle_damage_mean_region_code']], on='region_code', how='left')

In [ ]:
df2.sample(5)

# 3.0 Exploratory Data Analysis (EDA)

- Interest should occur with customer's previous insured
- Interest should occur with older people (more than 40 years)
- Interest should occur with new vehicles (less than 5 years) 
- Interest should occur with older customers (more then 150 days)
- Interest should occur with customers vehicles previously damaged
- Interest should occur with women
- Interest should occur with customers that spend more than 30k/year with health insurance
- Drivers license is greater in older people (more than 40 years)
- Previously Insured is greater in older people (more than 40 years)
- Vehicle Age is greater (more than 2 years) in older people (more than 40 years)
